In [ ]:
class StopExecution(Exception):
    def _render_traceback_(self):
        return []

In [ ]:
# Starting from Python 3.8 DLL search policy has changed.
# We need to add path to CUDA DLLs explicitly.
import os

if os.name == "nt":
    # Add CUDA_PATH env variable
    cuda_path = os.environ["CUDA_PATH"]
    if cuda_path:
        os.add_dll_directory(os.path.join(cuda_path, "bin"))
    else:
        raise StopExecution

In [ ]:
from PIL import Image
from IPython.display import display

import python_vali as vali
import numpy as np

In [ ]:
url = "../tests/data/test.mp4"

In [ ]:
# GPU-accelerated decoder
pyDec = vali.PyDecoder(
    url,
    {},
    gpu_id=0)

# GPU-accelerated converter
pyCvt = vali.PySurfaceConverter(
    pyDec.Format,
    vali.PixelFormat.RGB,
    gpu_id=0)

# GPU-accelerated Surface downloader
pyDwn = vali.PySurfaceDownloader(gpu_id=0)

In [ ]:
# Raw decoded Surface
surf_src = vali.Surface.Make(
    format=pyDec.Format,
    width=pyDec.Width,
    height=pyDec.Height,
    gpu_id=0)

# Raw Surface, converted to RGB
surf_dst = vali.Surface.Make(
    format=vali.PixelFormat.RGB,
    width=pyDec.Width,
    height=pyDec.Height,
    gpu_id=0)

# Numpy array which contains decoded RGB Surface
frame = np.ndarray(
    dtype=np.uint8,
    shape=(surf_dst.HostSize))

# Packet data
pkt_data = vali.PacketData()

while True:
    # Decode single Surface
    success, details = pyDec.DecodeSingleSurface(surf_src, pkt_data)
    if not success:
        print(details)
        raise StopExecution

    # Convert tot RGB
    success, details = pyCvt.Run(surf_src, surf_dst)
    if not success:
        print(details)
        raise StopExecution

    # Copy pixels to numpy ndarray
    success, details = pyDwn.Run(surf_dst, frame)
    if not success:
        print(details)
        raise StopExecution

    # Reshape to proper dimensions.
    # Some video formats like NV12 require to reshape to 'raw' dimensions.
    # Others like RGB24 require to reshape to 'final' dimensions. Hence this
    # feature isn't yet supported by VALI out of the box. Have to do that by
    # hand. Sic !
    res_frame = np.reshape(
        frame,
        (pyDec.Height, pyDec.Width, 3))

    # Display in notebook
    display(Image.fromarray(res_frame), display_id="decoded_frame")

    # Output packet data
    print(pkt_data)